In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")

from hc_nlp.pipeline import ThesaurusMatcher, EntityFilter, MapEntityTypes, DateMatcher
from hc_nlp.model_testing import test_ner
from hc_nlp.io import load_text_and_annotations_from_labelstudio
from hc_nlp.spacy_helpers import display_manual_annotations, display_ner_annotations
from hc_nlp import constants

import pprint
pp = pprint.PrettyPrinter(indent=2)

import pandas as pd
import spacy
import time

In [8]:
# LOAD MODELS

# pure NER
nlp_lg = spacy.load("en_core_web_lg")

# custom components
thes = ThesaurusMatcher(nlp_lg, thesaurus_path="../data/labels_all_unambiguous_types_people_orgs.jsonl", case_sensitive=False)
thes_ow = ThesaurusMatcher(nlp_lg, thesaurus_path="../data/labels_all_unambiguous_types_people_orgs.jsonl", 
                                  case_sensitive=False, overwrite_ents=True)
entityfilter = EntityFilter(ignore_ent_labels=['DATE'])

# model with people & orgs thesaurus annotations before model
nlp_thes = spacy.load("en_core_web_lg")
nlp_thes.add_pipe(thes, before='ner')
nlp_thes.add_pipe(entityfilter, last=True)

# model with people & orgs thesaurus annotations after model
nlp_thes_aft = spacy.load("en_core_web_lg")
nlp_thes_aft.add_pipe(thes, after='ner')
nlp_thes_aft.add_pipe(entityfilter, last=True)

# model with people & orgs thesaurus annotations after model, overwriting NER annotations
nlp_thes_aft_ow = spacy.load("en_core_web_lg")
nlp_thes_aft_ow.add_pipe(thes_ow, after='ner')
nlp_thes_aft_ow.add_pipe(entityfilter, last=True)

# model with rules for dates
datematcher = DateMatcher(nlp_lg)
nlp_dates_bef = spacy.load("en_core_web_lg")
nlp_dates_bef.add_pipe(datematcher, before='ner')

# model with rules for dates before & thesaurus after
nlp_dates_bef_thes_aft = spacy.load("en_core_web_lg")
nlp_dates_bef_thes_aft.add_pipe(datematcher, before='ner')
nlp_dates_bef_thes_aft.add_pipe(thes, after='ner')
nlp_dates_bef_thes_aft.add_pipe(entityfilter, last=True)

# model with rules for dates before & thesaurus after, with overwrite
nlp_dates_bef_thes_aft_ow = spacy.load("en_core_web_lg")
nlp_dates_bef_thes_aft_ow.add_pipe(datematcher, before='ner')
nlp_dates_bef_thes_aft_ow.add_pipe(thes_ow, after='ner')
nlp_dates_bef_thes_aft_ow.add_pipe(entityfilter, last=True)

# add mapping from Spacy to HC types to all pipelines
mapentitytypes = MapEntityTypes(nlp_lg, validate_mapping=False)
nlp_lg.add_pipe(mapentitytypes)
nlp_thes.add_pipe(mapentitytypes)
nlp_thes_aft.add_pipe(mapentitytypes)
nlp_thes_aft_ow.add_pipe(mapentitytypes)
nlp_dates_bef.add_pipe(mapentitytypes)
nlp_dates_bef_thes_aft.add_pipe(mapentitytypes)
nlp_dates_bef_thes_aft_ow.add_pipe(mapentitytypes)

nlp_lg.pipe_names, nlp_thes.pipe_names, nlp_thes_aft.pipe_names, nlp_thes_aft_ow.pipe_names, nlp_dates_bef.pipe_names, nlp_dates_bef_thes_aft.pipe_names, nlp_dates_bef_thes_aft_ow.pipe_names


2020-12-10 16:41:21,344 - hc_nlp.pipeline - INFO - Loading thesaurus from ../data/labels_all_unambiguous_types_people_orgs.jsonl
2020-12-10 16:41:21,344 - hc_nlp.pipeline - INFO - Loading thesaurus from ../data/labels_all_unambiguous_types_people_orgs.jsonl
2020-12-10 16:41:32,707 - hc_nlp.pipeline - INFO - 17016 term thesaurus imported in 11s
2020-12-10 16:41:32,707 - hc_nlp.pipeline - INFO - 17016 term thesaurus imported in 11s
2020-12-10 16:41:32,709 - hc_nlp.pipeline - INFO - Loading thesaurus from ../data/labels_all_unambiguous_types_people_orgs.jsonl
2020-12-10 16:41:32,709 - hc_nlp.pipeline - INFO - Loading thesaurus from ../data/labels_all_unambiguous_types_people_orgs.jsonl
2020-12-10 16:41:36,189 - hc_nlp.pipeline - INFO - 17016 term thesaurus imported in 3s
2020-12-10 16:41:36,189 - hc_nlp.pipeline - INFO - 17016 term thesaurus imported in 3s


(['tagger', 'parser', 'ner', 'MapEntityTypes'],
 ['tagger',
  'parser',
  'ThesaurusMatcher',
  'ner',
  'EntityFilter',
  'MapEntityTypes'],
 ['tagger',
  'parser',
  'ner',
  'ThesaurusMatcher',
  'EntityFilter',
  'MapEntityTypes'],
 ['tagger',
  'parser',
  'ner',
  'ThesaurusMatcher',
  'EntityFilter',
  'MapEntityTypes'],
 ['tagger', 'parser', 'DateMatcher', 'ner', 'MapEntityTypes'],
 ['tagger',
  'parser',
  'DateMatcher',
  'ner',
  'ThesaurusMatcher',
  'EntityFilter',
  'MapEntityTypes'],
 ['tagger',
  'parser',
  'DateMatcher',
  'ner',
  'ThesaurusMatcher',
  'EntityFilter',
  'MapEntityTypes'])

In [9]:
data = load_text_and_annotations_from_labelstudio("../labelling/export/2020-12-10-12-43-04.zip", nlp_lg)

len(data)

526

In [10]:
def generate_results_table(pipelines: dict, examples: list):
    results = pd.DataFrame()
    times = dict()
    for name, p in pipelines.items():
        start = time.time()
        temp_res = pd.json_normalize(test_ner(p, examples=examples))
        end = time.time()
        temp_res['pipeline'] = name
        
        results = results.append(temp_res)
        times[name] = end-start
        
        print(f"{name}: {int(end-start)}s")
    
    results = results.set_index('pipeline')
    
    return results

res = generate_results_table(
    {
        'NER': nlp_lg, 
        'NER_thes_bef': nlp_thes, 
        'NER_thes_aft': nlp_thes_aft, 
        'NER_thes_aft_ow': nlp_thes_aft_ow,
        'NER_datematcher_bef': nlp_dates_bef,
        'NER_datematcher_bef_thes_aft': nlp_dates_bef_thes_aft,
        'NER_datematcher_bef_thes_aft_ow': nlp_dates_bef_thes_aft_ow
#         'NER_datematcher_aft': nlp_dates_aft,
    }, 
    data)

res.T
        

2020-12-10 16:42:38,810 - hc_nlp.model_testing - WARNING - Poster, London & North Eastern Railway, Bamburgh by Tom Purvis, 1936. Coloured lithograph depicting a stylised view of the coast with Bamburgh castle, the beach and village. Printed by Chorley & Pickersgill Ltd, Lithographers, Leeds. Format: double royal. Dimensions: 40 x 25 inches, 1016 x 635mm. skipped
2020-12-10 16:42:39,361 - hc_nlp.model_testing - WARNING - Glass bottle containing unknown grey, metallic-looking powder. Part of Statham's student chemical laboratory. skipped
2020-12-10 16:42:39,816 - hc_nlp.model_testing - WARNING - Bone gouge probably by McQueen of Newcastle, England, late 19th early 20th century, nickel plated steel, handle probably brass, nickel plated skipped
2020-12-10 16:42:40,213 - hc_nlp.model_testing - WARNING - print. lithograph, tinted. Britannia Tubular Bridge over the Menai Straits, taken during its construction in 1849... from the Angelsea Shore.../G. Hawkins. printed and published Day & Son, 1

pipeline,NER,NER_thes_bef,NER_thes_aft,NER_thes_aft_ow,NER_datematcher_bef,NER_datematcher_bef_thes_aft,NER_datematcher_bef_thes_aft_ow
ents_p,51.3795,56.25,55.9517,56.738,52.3932,57.0131,57.7955
ents_r,67.7885,67.3764,66.8269,67.3764,69.9176,68.956,69.5055
ents_f,58.4542,61.3125,60.9077,61.6013,59.9,62.4184,63.1119
support,1549,1549,1549,1549,1549,1549,1549
labels_missing_from_annotations,"[ORDINAL, CARDINAL, LANGUAGE, PERCENT, LAW, FA...","[ORDINAL, CARDINAL, LANGUAGE, PERCENT, LAW, FA...","[ORDINAL, CARDINAL, LANGUAGE, PERCENT, LAW, FA...","[ORDINAL, CARDINAL, LANGUAGE, PERCENT, LAW, FA...","[ORDINAL, CARDINAL, LANGUAGE, LAW, FAC, MONEY,...","[ORDINAL, CARDINAL, LANGUAGE, LAW, FAC, MONEY,...","[ORDINAL, CARDINAL, LANGUAGE, LAW, FAC, MONEY,..."
ents_per_type.LOC.p,78.2895,81.8815,80.339,82.5175,78.2895,80.339,82.5175
ents_per_type.LOC.r,79.5987,78.5953,79.2642,78.9298,79.5987,79.2642,78.9298
ents_per_type.LOC.f,78.9386,80.2048,79.798,80.6838,78.9386,79.798,80.6838
ents_per_type.LOC.support,322,322,322,322,322,322,322
ents_per_type.ORG.p,51.0917,50.303,51.7564,51.1294,51.0917,51.7564,51.1294


In [11]:
# res.T.to_csv("../data/AAIL_results_nlp_lg_2.csv")

In [7]:
for text, annotations in data[0:50]:
#     print("NER")
#     display_ner_annotations(text, nlp_lg)
    print("thes people & orgs")
    display_ner_annotations(text, nlp_thes)
    print([(ent.text, ent.ent_id_) for ent in nlp_thes(text).ents])
    print("thes people & orgs (after)")
    display_ner_annotations(text, nlp_thes_aft)
    print([(ent.text, ent.ent_id_) for ent in nlp_thes_aft(text).ents])
    print("GT")
    display_manual_annotations(text, annotations)
    print("-----")
    print("-----")

thes people & orgs


[('Newhaven', ''), ('Dieppe', ''), ('C Mc D Mann & Co Ltd', ''), ('Hanley', ''), ('44 mm', ''), ('0.73kg', '')]
thes people & orgs (after)


[('Newhaven', ''), ('Dieppe', ''), ('C Mc D Mann & Co Ltd', ''), ('Hanley', ''), ('44 mm', ''), ('0.73kg', '')]
GT


-----
-----
thes people & orgs


[('c.1930-1969', ''), ('Smith', 'https://collection.sciencemuseumgroup.org.uk/people/cp120932'), ('Britain', ''), ('569', '')]
thes people & orgs (after)


...

[('British', ''), ('Wembley', '')]
GT


-----
-----
thes people & orgs


[('John Pollock & Co.', '')]
thes people & orgs (after)


[('John Pollock & Co.', '')]
GT


-----
-----
